In [34]:
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from joblib import dump, load
from os import path
from sklearn import metrics

In [35]:
time_precision = 4   # jagame päeva xh suurusteks plokkideks


X_train = pd.read_csv("../../data/X_train.csv")
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.ToimAeg = X_train.ToimAeg.values.astype('datetime64[D]')
X_train.ToimAeg = X_train['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_train["Aasta"] = X_train['ToimAeg'].dt.year
X_train["Kuu"] = X_train['ToimAeg'].dt.month
X_train["Päev"] = X_train['ToimAeg'].dt.day
X_train["Tund"] = X_train['ToimAeg'].dt.hour
X_train.drop(columns="ToimAeg", inplace=True)
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.head()

y_train = pd.read_csv("../../data/y_train.csv")
y_train = y_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_train.head()

X_valid = pd.read_csv("../../data/X_valid.csv")
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.ToimAeg = X_valid.ToimAeg.values.astype('datetime64[D]')
X_valid.ToimAeg = X_valid['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_valid["Aasta"] = X_valid['ToimAeg'].dt.year
X_valid["Kuu"] = X_valid['ToimAeg'].dt.month
X_valid["Päev"] = X_valid['ToimAeg'].dt.day
X_valid["Tund"] = X_valid['ToimAeg'].dt.hour
X_valid.drop(columns="ToimAeg", inplace=True)
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.head()

y_valid = pd.read_csv("../../data/y_valid.csv")
y_valid = y_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_valid.head()

X_train.head()

y_train.info()
y_train.drop(columns=["Kahjusumma","Koht"], inplace=True)
y_valid.drop(columns=["Kahjusumma", "Koht"], inplace=True)
y_train.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62476 entries, 0 to 62475
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LIIK_AVALIKU_KORRA_RIKKUMINE   62476 non-null  int8  
 1   LIIK_JALGRATTA_MOPEEDI_VARGUS  62476 non-null  int8  
 2   LIIK_KEHALINE_VAARKOHTLEMINE   62476 non-null  int8  
 3   LIIK_KELMUS                    62476 non-null  int8  
 4   LIIK_KONFLIKT/OMAVOLI          62476 non-null  int8  
 5   LIIK_KOOLIVAGIVALD             62476 non-null  int8  
 6   LIIK_LIIKLUSRIKKUMINE          62476 non-null  int8  
 7   LIIK_MOOTORSOIDUKI_VARGUS      62476 non-null  int8  
 8   LIIK_MUU                       62476 non-null  int8  
 9   LIIK_PISIVARGUS                62476 non-null  int8  
 10  LIIK_ROOVIMINE                 62476 non-null  int8  
 11  LIIK_TULEKAHJU                 62476 non-null  int8  
 12  LIIK_VANDALISM                 62476 non-null  int8  
 13  L

,LIIK_AVALIKU_KORRA_RIKKUMINE,LIIK_JALGRATTA_MOPEEDI_VARGUS,LIIK_KEHALINE_VAARKOHTLEMINE,LIIK_KELMUS,LIIK_KONFLIKT/OMAVOLI,LIIK_KOOLIVAGIVALD,LIIK_LIIKLUSRIKKUMINE,LIIK_MOOTORSOIDUKI_VARGUS,LIIK_MUU,LIIK_PISIVARGUS,LIIK_ROOVIMINE,LIIK_TULEKAHJU,LIIK_VANDALISM,LIIK_VARGUS
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [36]:
xgb_models = list()
for col in y_train.columns:
    xgb_models.append(XGBRegressor().fit(X_train, y_train[col]))

In [37]:
for col, model in list(zip(y_valid.columns, xgb_models)):
    print(metrics.roc_auc_score(y_valid[col], model.predict(X_valid)), col, sep="\t")

0.9328272382272141	LIIK_AVALIKU_KORRA_RIKKUMINE
0.8943459082534883	LIIK_JALGRATTA_MOPEEDI_VARGUS
0.8897460942494054	LIIK_KEHALINE_VAARKOHTLEMINE
0.8033317561942637	LIIK_KELMUS
0.721415544332211	LIIK_KONFLIKT/OMAVOLI
0.9399610460200508	LIIK_KOOLIVAGIVALD
0.6470801828356615	LIIK_LIIKLUSRIKKUMINE
0.8352539352539353	LIIK_MOOTORSOIDUKI_VARGUS
0.7483144691000476	LIIK_MUU
0.8771679077550626	LIIK_PISIVARGUS
0.7456011208278636	LIIK_ROOVIMINE
0.5242338761379663	LIIK_TULEKAHJU
0.8607412623541656	LIIK_VANDALISM
0.7732725160810157	LIIK_VARGUS


In [38]:
# find best cutoff
def applyCutoff(df, cutoff):
    a = df.copy()
    for col in a.columns:
        a[col] = a[col].apply(lambda x: 1 if x > cutoff else 0)
    return a
max_acc = {col:-1 for col in y_valid.columns}
max_acc_cut = {col:-1 for col in y_valid.columns}
for col, model in list(zip(y_valid.columns, xgb_models)):
    temp = pd.DataFrame(model.predict(X_valid))
    for i in np.arange(0.3, 0.8, 0.01):
        temp_cut = applyCutoff(temp, i)
        if metrics.accuracy_score(y_valid[col], temp_cut) > max_acc[col]:
            max_acc[col] = metrics.accuracy_score(y_valid[col], temp_cut)
            max_acc_cut[col] = i
print(max_acc)
print(max_acc_cut)

{'LIIK_AVALIKU_KORRA_RIKKUMINE': 0.950313740555769, 'LIIK_JALGRATTA_MOPEEDI_VARGUS': 0.9760532718657958, 'LIIK_KEHALINE_VAARKOHTLEMINE': 0.9683698296836983, 'LIIK_KELMUS': 0.9946215904725317, 'LIIK_KONFLIKT/OMAVOLI': 0.9957741067998463, 'LIIK_KOOLIVAGIVALD': 0.9987194263029837, 'LIIK_LIIKLUSRIKKUMINE': 0.9992316557817903, 'LIIK_MOOTORSOIDUKI_VARGUS': 0.994877705211935, 'LIIK_MUU': 0.9632475348956332, 'LIIK_PISIVARGUS': 0.8140606991932385, 'LIIK_ROOVIMINE': 0.9882187219874504, 'LIIK_TULEKAHJU': 0.9987194263029837, 'LIIK_VANDALISM': 0.9288001024458957, 'LIIK_VARGUS': 0.6982968369829684}
{'LIIK_AVALIKU_KORRA_RIKKUMINE': 0.48000000000000015, 'LIIK_JALGRATTA_MOPEEDI_VARGUS': 0.5000000000000002, 'LIIK_KEHALINE_VAARKOHTLEMINE': 0.5600000000000003, 'LIIK_KELMUS': 0.7300000000000004, 'LIIK_KONFLIKT/OMAVOLI': 0.7600000000000005, 'LIIK_KOOLIVAGIVALD': 0.3, 'LIIK_LIIKLUSRIKKUMINE': 0.36000000000000004, 'LIIK_MOOTORSOIDUKI_VARGUS': 0.5300000000000002, 'LIIK_MUU': 0.7700000000000005, 'LIIK_PISIVARGU

In [39]:
# best cutoff is where sum(frequency*cutof)/all
summa = 0
for col in y_valid.columns:
    summa += max_acc_cut[col] * sum(y_valid[col])
print(summa/y_valid.shape[0])
# best is 0.6015840696632094, although we can use separate cutoffs for everyone, but to lazy

0.6015840696632094


In [40]:
# accuracies
for col, model in list(zip(y_valid.columns, xgb_models)):
    print(metrics.accuracy_score(y_valid[col], applyCutoff(pd.DataFrame(model.predict(X_valid)), max_acc_cut[col])), col, sep="\t")

0.950313740555769	LIIK_AVALIKU_KORRA_RIKKUMINE
0.9760532718657958	LIIK_JALGRATTA_MOPEEDI_VARGUS
0.9683698296836983	LIIK_KEHALINE_VAARKOHTLEMINE
0.9946215904725317	LIIK_KELMUS
0.9957741067998463	LIIK_KONFLIKT/OMAVOLI
0.9987194263029837	LIIK_KOOLIVAGIVALD
0.9992316557817903	LIIK_LIIKLUSRIKKUMINE
0.994877705211935	LIIK_MOOTORSOIDUKI_VARGUS
0.9632475348956332	LIIK_MUU
0.8140606991932385	LIIK_PISIVARGUS
0.9882187219874504	LIIK_ROOVIMINE
0.9987194263029837	LIIK_TULEKAHJU
0.9288001024458957	LIIK_VANDALISM
0.6982968369829684	LIIK_VARGUS


In [41]:
for col, model in list(zip(y_valid.columns, xgb_models)):
    print(sum(model.predict(X_valid)), col, sep="\t")


844.8389670923352	LIIK_AVALIKU_KORRA_RIKKUMINE
191.8310616733288	LIIK_JALGRATTA_MOPEEDI_VARGUS
261.7957282946518	LIIK_KEHALINE_VAARKOHTLEMINE
42.44715542541962	LIIK_KELMUS
24.8134399865221	LIIK_KONFLIKT/OMAVOLI
14.23281026765062	LIIK_KOOLIVAGIVALD
11.077009453512233	LIIK_LIIKLUSRIKKUMINE
45.92504049169656	LIIK_MOOTORSOIDUKI_VARGUS
300.601483178536	LIIK_MUU
2350.5140913507203	LIIK_PISIVARGUS
83.38235109356174	LIIK_ROOVIMINE
13.523699544922692	LIIK_TULEKAHJU
562.3993943728638	LIIK_VANDALISM
3781.197805321688	LIIK_VARGUS


In [42]:
print(4/0)
# dump theese model
for col, model in list(zip(y_valid.columns, xgb_models)):
    filename = col.replace("/", "")
    name = path.join("trained_models/xgbr/", "xgbr_"+filename+".joblib")
    dump(model, filename=name, compress=3, protocol=5)

ZeroDivisionError: division by zero